<a href="https://colab.research.google.com/github/ftvalentini/misc-notebooks/blob/master/borges_hipalages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buscando hipálages en los cuentos de Borges con NLP
#### Autores: Julián Martínez Correa y Francisco Valentini

En Twitter alguién sugirió recopilar todas las hipálages de la literatura de Borges y [@manuelaristaran sugirió usar alguna técnica de Natural Language Processing](https://twitter.com/manuelaristaran/status/1249775645929934851) para esta tarea.  
Una hipálage es una combinación de palabras en la que se le atribuye a un sustantivo una acción (verbo) o una cualidad (adjetivo) que son propios de otro. Por ejemplo, la expresión "noche unámine" en el Las Ruinas Circulares.  
Siguiendo la sugerencia, decidimos abordar de forma exploratoria esta tarea con los siguientes pasos:  
- Restringimos el análisis a pares de palabras sustantivo-adjetivo y a los cuentos de Ficciones **en inglés**
- Extraemos todos los pares sustantivo-adjetivo de los cuentos usando un [modelo de dependecy parsing pre-entrenado](https://stanfordnlp.github.io/stanza/depparse.html) que reconoce las dependencias gramáticales en cada oración
- Medimos las distancias coseno entre los [word embeddings GloVe](https://nlp.stanford.edu/projects/glove/) de cada par de palabras
- Rankeamos los pares según la distancia coseno y exploramos un ranking alternativo que normaliza la distancia por la distancia promedio del sustantivo o adejtivo a los *n* adjetivos o sustantivos más cercanos.

La hipótesis es que hay una mayor probabilidad de encontrar hipálages en pares sustatntivo-adjetivo con distancia alta entre sus embeddings que en pares con distancia baja. Esto se debe a que **en general** las palabras que conforman las hipálages tienen una similitud semántica baja o al menos no tienden a usarse en el mismo contexto. 

En este sentido, este análisis no nos permite estricamente encontrar hipálages (y aún menos evaluar la bondad de los resultados), sino más bien encontrar pares de sustantivo-adjetivos inusuales, llamativos o interesantes en relación al uso común de la lengua inglesa.

### Descargas y librerías

- Instalamos [Stanza](https://github.com/stanfordnlp/stanza/), la librería desarrollada por Stanford que vamos a usar para encontrar las dependencias gramaticales en los textos. La librería ofrece modelos pre-entrenados para múltiples idiomas para aplicar en el contexto de tareas de NLP (en nuestro caso, dependency parsing). 

In [0]:
!pip install stanza

- Descargamos `PyPDF2` para convertir *Ficciones* de formato PDF a un archivo de texto plano.  

In [0]:
!pip install PyPDF2

- Descargamos los modelos de `stanza` para textos en inglés. Los modelos se guardan por defecto en `/root/stanza_resources`. Esta carga nos llevó poco tiempo porque usamos Google Colab,  pero en otro contexto puede llevar algunos minutos -- lo mismo vale para el resto de instalaciones o descargas.

In [0]:
### PUEDE TARDAR VARIOS MINUTOS ###
stanza.download('en')

- Cargamos las librerías que vamos a necesitar para hacer el análisis.

In [0]:
import numpy as np
import pandas as pd

import stanza

import nltk
from nltk.tokenize import sent_tokenize
import gensim.downloader as gensim_api

import PyPDF2 

- Cargamos los vectores [GloVe](https://nlp.stanford.edu/pubs/glove.pdf), word embeddings ajustados por Stanford. En particular, vamos a usar los vectores de mayor dimensión posible (300) y ajustados con la wikipedia en inglés.

In [0]:
def load_embeddings():
    """ Load GloVe Vectors
        Returns: array of size (vocab_size, embeddings_dim)
    """
    vectors = gensim_api.load("glove-wiki-gigaword-300")
    return vectors

In [0]:
### PUEDE TARDAR VARIOS MINUTOS ###
word_vectors = load_embeddings()

- Descargamos los modelos de tokenización `punkt` porque son necesarios para separar el texto en oraciones.

In [0]:
## revisar si esto es necesario
nltk.download('punkt')

- Convertimos el pdf con los cuentos en un archivo de texto. Leemos este archivo y lo almacenamos en `texto` como una gran cadena de caracteres.

In [0]:
### ACLARAR CUANTO TARDA
def pdf_to_txt(pdf_file, txt_file):
    """ Convert pdf_file to txt_file with strings
    """
    with open(pdf_file,'rb') as pdf, open(txt_file, 'w') as txt:
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        number_of_pages = read_pdf.getNumPages()
        for page_number in range(number_of_pages):  
            page = read_pdf.getPage(page_number)
            page_content = page.extractText()
            text_file.write(page_content)

pdf_to_txt('borges_collected_fictions.pdf', 'ficciones.txt')
with open('ficciones.txt', "r", encoding='utf-8') as f:
    texto = f.read() 

### Limpieza del texto

Limpiamos el texto de la siguiente manera:  
- Reemplazamos los saltos de linea y cualquier otro carácter de tipo whitespace por espacios no repetidos porque no nos interesan los cambios de párrafo ni de página, sino solamente las oraciones  

La documentación de `stanza` sugiere juntar varias oraciones en batches separando cada oración con dos saltos de linea `\n\n` para acelerar el procesamiento de muchos documentos. Sin embargo esto dificultaría rastrear al final del análisis las oraciones donde se encuentran las potenciales hipálages -- decidimos entonces conservar el texto como un solo string y que `stanza` se encargue de la tokenización.  

In [0]:
def clean_texto(texto):
    """Cleans raw text
       Returns: clean string
    """
    clean_text = " ".join(texto.split())
    return clean_text

texto = clean_texto(texto)

Veamos los 300 caracteres del texto separados en oraciones para facilitar la visualización:

In [0]:
# REVISAR QUE FUNCIONE
for s in sent_tokenize(texto[:300]):
    print(s)

### Dependencies parsing

Para realizar tareas de NLP con `stanza` es necesario construir un Pipeline con procesadores acordes al idioma de análisis.
El Pipeline se inicializa por defecto con un dependency parser (`depparse`) para determinar las dependencias sintácticas entre las palabras de una oración dada. Para poder correr este modelo es necesario procesar el texto de antemano con los siguientes procesadores:
- `tokenize`: separa el documento en oraciones
- `pos`: identifica el rol de cada palabra en la oración con un modelo de Part-of-Speech (POS) tagging pre-entrenado
- `lemma`: halla el lema correspondiente de cada palabra (la forma que representa las posibles formas flexionadas -- por ejemplo, "walk" es el lema de "walking")

Por defecto estos procesadores se ejecutan antes de `depparse`, el cual usa la información generada en los pasos precedentes para identificar las relaciones sintátictas. Para identificar las relaciones sustantivo-adjetivo es necesario conservar las relaciones de tipo "amod" (*adjectival modifier*).

In [0]:
nlp = stanza.Pipeline('en', 'tokenize,pos,lemma,depparse', verbose=False)
docs = nlp(texto)

def find_dependencies(doc):
    """Find amod dependencies in one parsed doc with one or more sentences 
    Returns: list of dependencies with (sentence_idx, head, dependent)
    """
    deps = list()
    for sent_idx, sent in enumerate(doc.sentences):
        id2word = {word.id: word.text for word in sent.words}
        deps += [(sent_idx, id2word[str(word.head)], word.text) \
                for word in sent.words if word.deprel=='amod']
    return deps
    
deps = find_dependencies(docs)

## Word embeddings

Calculamos las distancias entre los embeddings de las palabras que forman la dependencia y luego generamos un DataFrame para guardarlas

In [0]:
vocab = word_vectors.vocab.keys()
deps_lists = [list(d) for d in deps]
for dep in deps_lists:
    _words = dep[1:3] 
    if set(_words).issubset(vocab):
        dep.append(word_vectors.distance(*_words))

In [0]:
data_dep = pd.DataFrame(deps_lists, columns =['sent_idx', 'head', 'dep', 'dist']) 

*Agrega* distancia con respecto a palabra más cercana
(despues: modificar para que sea un adjetivo / sustantivo!)

In [0]:
def most_similar(word, n):
    """Return nth most similar word and distance"""
    if word in vocab:
        out = word_vectors.most_similar(word, topn=20)[n+1]
    else:
        out = (np.nan, np.nan)
    return out

In [0]:
def get_upos(palabra):
    doc = nlp(palabra)
    upos = doc.sentences[0].words[0].upos
    return upos
def distance_most_similar_adjs(tmp, topn=5):
    distancias = list()
    k = 0
    for i in tmp:
        if get_upos(i[0]) == 'ADJ':
            out.append(i[1])
            k += 1
        if k == topn:
            break
    avg_distance = sum(distancias) / len(distancias)
    return avg_distance
def distance_most_similar(tmp, type='ADJ', topn=5):
    """type 'ADJ' or 'NOUN'
    """
    distancias = list()
    k = 0
    for i in tmp:
        if get_upos(i[0]) == type:
            out.append(i[1])
            k += 1
        if k == topn:
            break
    avg_distance = sum(distancias) / len(distancias)
    return avg_distance

In [0]:
data_dep[['dep_closest_word','dep_closest_dist']] = \
    data_dep.apply(lambda x: pd.Series(most_similar(x['dep'], n=1)), axis=1)
data_dep[['head_closest_word','head_closest_dist']] = \
    data_dep.apply(lambda x: pd.Series(most_similar(x['head'], n=1)), axis=1)

In [0]:
data_dep['ratio_dep'] = data_dep['dist'] / data_dep['dep_closest_dist']
data_dep['ratio_head'] = data_dep['dist'] / data_dep['head_closest_dist']

In [0]:
# exploramos
# data_dep.sort_values(by=['dist'], inplace=True, ascending=False)
# data_dep.sort_values(by=['ratio_dep'], inplace=True, ascending=False)
data_dep.sort_values(by=['ratio_head'], inplace=True, ascending=False)
df_top = data_dep.head(20).copy()
df_top['sentence'] = df_top['sent_idx'].apply(lambda x: docs_.sentences[x].text)
df_bottom = data_dep.tail(20).copy()
df_bottom['sentence'] = df_bottom['sent_idx'].apply(lambda x: docs_.sentences[x].text)

In [0]:
for i,j,k in zip(df_top['dep'], df_top['head'], df_top['sentence']):
    print(i,j,' -- ',k)

In [0]:
print(df_bottom[['dep','head','dist']])

In [0]:
# ojaldre con estas cosas:
'Library' in vocab

*Cosas viejas*

In [0]:
lengths = [len(doc.sentences) for doc in docs]
for i in lengths:
    print(i)
# No siempre da 1!!! (pero se puede corregir el idx de abajo creo)
print(deps[len(deps)-1])
sentences[129]
print(deps[:5])
# funciona el rastreo de oraciones!! :)

# def words_distance(vocab, *words):
#     """ Get (cosine?) distance between 2 Glove word vectors if in vocabulary \ 
#         (dict_keys) 
#     Returns: distance (float)
#     """
#     if set(words).issubset(vocab):
#         d = word_vectors.distance(*words)
#     else:
#         d = np.nan
#     return d
# vocab = word_vectors.vocab.keys()
# data_dep = pd.DataFrame(deps, columns =['sent_idx', 'head', 'dep']) 
# data_dep['dist'] = data_dep.apply(lambda x: \
#                                   words_distance(vocab, x['head'], x['dep']), 
#                                   axis=1)

In [0]:
### OLD CODE

# def find_dependencies_v2(docs):
#     """Find amod dependencies in many parsed docs (idx of sentence with \
#     respect to all docs)
#     Returns: list of dependencies with (sentence_idx, head, dependent)
#     """
#     deps = list()
#     sent_idx = 0
#     for doc in docs:
#         for sent in doc.sentences:
#             id2word = {word.id: word.text for word in sent.words}
#             deps += [(sent_idx, id2word[str(word.head)], word.text) \
#                     for word in sent.words if word.deprel=='amod']
#         sent_idx += 1
#     return deps
# deps = find_dependencies(docs)

# def create_batches(sentences, sentences_per_doc=16):
#     """Creates batches of sentences
#        Returns: list of strings, each one containg multiple sentences separated
#        by \n\n 
#     """
#     ranges = [(max(0,i), min(i+sentences_per_doc,len(sentences))) \
#                  for i, x in enumerate(sentences) if \
#                  i % sentences_per_doc == 0]
#     batches = ['\n\n'.join(sentences[i:s]) for i, s in ranges]
#     return batches

# sentences = clean_texto(texto)
# batches = create_batches(sentences, sentences_per_doc=16)
# print('{} sentences found'.format(len(sentences)))
# print('{} batches created'.format(len(batches)))

# def parse_sentences(batches):
#     """Parse batched sentences 
#     Returns: a list of docs, each one cointaining multiple parsed sentences
#     """
#     docs = list()
#     for doc in batches:
#         docs.append(nlp(doc))
#     # ponemos los docs en una sola lista
#     docs = list(docs)
#     return docs
# # docs = parse_sentences(batches)
# docs = parse_sentences(sentences)